In [1]:
import os

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'
USERNAME = os.environ['SASL_PLAIN_USERNAME']
PASSWORD = os.environ['SASL_PLAIN_PASSWORD']
options = {
    "kafka.sasl.jaas.config": f'org.apache.kafka.common.security.plain.PlainLoginModule required username="{USERNAME}" password="{PASSWORD}";',
    "kafka.sasl.mechanism": "PLAIN",
    "kafka.security.protocol" : "SASL_SSL",
    "kafka.bootstrap.servers": 'pkc-lzoyy.europe-west6.gcp.confluent.cloud:9092',
    "group.id": "spark_consumer",
}

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark = SparkSession \
    .builder \
    .appName("Spark-Notebook") \
    .getOrCreate()

23/03/13 19:27:09 WARN Utils: Your hostname, LX10001 resolves to a loopback address: 127.0.1.1; using 192.168.1.8 instead (on interface wlp0s20f3)
23/03/13 19:27:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/mrmarx/Downloads/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/mrmarx/.ivy2/cache
The jars for the packages stored in: /home/mrmarx/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9aaab6cf-af2d-422f-9fb9-540f68c035d4;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org

23/03/13 19:27:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
ride_schema = T.StructType(
    [
         T.StructField('pickup_datetime', T.StringType()),
         T.StructField('dropOff_datetime', T.StringType()),
         T.StructField("PUlocationID", T.IntegerType()),
         T.StructField("DOlocationID", T.IntegerType()),
     ])

In [5]:
for topic in ['rides_green', 'rides_fhv']:

    df_kafka_raw = spark \
        .readStream \
        .format("kafka") \
        .option("subscribe", topic) \
        .option("startingOffsets", "earliest") \
        .option("checkpointLocation", "checkpoint") \
        .options(**options) \
        .load()


    df_kafka_encoded = df_kafka_raw.selectExpr("CAST(key AS STRING)","CAST(value AS STRING)")
    df = df_kafka_encoded.select(F.from_json(F.col("value").cast("string"),ride_schema).alias("value"))
    exploded_df = df.selectExpr('value.pickup_datetime', 'value.dropOff_datetime', 'value.PUlocationID', 'value.DOlocationID')
    agg_df = exploded_df.groupBy("PUlocationID").agg(F.count("*").alias("n_rides"))
    df_kafka = agg_df \
            .writeStream \
            .outputMode("complete") \
            .queryName(topic) \
            .format('memory') \
            .start()

23/03/13 19:27:18 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3d10d26c-6100-4357-be62-8462728cbce7. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/13 19:27:18 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/03/13 19:27:19 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-0be52564-b356-44c5-97ec-54d803eb1ee6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/03/13 19:27:19 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not support

In [6]:
spark.sql("""
    SELECT PUlocationID, SUM(n_rides) AS n_rides
    FROM (
    SELECT * FROM rides_green
        UNION ALL
    SELECT * FROM rides_fhv
    )
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 1
    """
).collect()

[Row(PUlocationID=74, n_rides=37759)]

## Most popular pickup location is 74